In [2]:
class Node:
    def __init__(self, name, phone):
        self.name = name
        self.phone = phone
        self.next = None

class Phonebook:
    def __init__(self):
        self.head = None

    def add_contact(self, name, phone):
        new_node = Node(name, phone)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def display_contacts(self):
        if self.head is None:
            print("Phonebook is empty.")
        else:
            current = self.head
            while current:
                print(f"Name: {current.name}, Phone: {current.phone}")
                current = current.next

# Example usage
phonebook = Phonebook()

while True:
    print("\n1. Add Contact")
    print("\n2. Display Contacts")
    print("\n3. Exit")
    choice = input("Enter your choice: ")

    if choice == '1':
        name = input("Enter Name: ")
        phone = input("Enter Phone Number: ")
        phonebook.add_contact(name, phone)
    elif choice == '2':
        phonebook.display_contacts()
    elif choice == '3':
        break
    else:
        print("Invalid choice! Please select a valid option.")



1. Add Contact

2. Display Contacts

3. Exit

1. Add Contact

2. Display Contacts

3. Exit

1. Add Contact

2. Display Contacts

3. Exit
Name: sameer srhianth, Phone: 9471877228
Name: sumit, Phone: 2347454

1. Add Contact

2. Display Contacts

3. Exit


In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, Scrollbar, END
import json
from PIL import Image, ImageTk

class Node:
    def __init__(self, name, phone):
        self.name = name
        self.phone = phone
        self.next = None

class Phonebook:
    def __init__(self):
        self.head = None

    def add_contact(self, name, phone):
        new_node = Node(name, phone)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def get_contacts(self):
        contacts = []
        current = self.head
        while current:
            contacts.append({"name": current.name, "phone": current.phone})
            current = current.next
        return contacts

    def find_contact(self, search_query):
        results = []
        current = self.head
        while current:
            if search_query.lower() in current.name.lower() or search_query in current.phone:
                results.append(f"{current.name}: {current.phone}")
            current = current.next
        return results

    def edit_contact(self, name, new_phone):
        current = self.head
        while current:
            if current.name == name:
                current.phone = new_phone
                return True
            current = current.next
        return False

    def delete_contact(self, name):
        if self.head and self.head.name == name:
            self.head = self.head.next
            return True
        current = self.head
        prev = None
        while current:
            if current.name == name:
                if prev:
                    prev.next = current.next
                else:
                    self.head = current.next
                return True
            prev = current
            current = current.next
        return False

    def load_contacts(self, contact_list):
        for contact in contact_list:
            self.add_contact(contact['name'], contact['phone'])

class PhonebookApp:
    def __init__(self, root):
        self.phonebook = Phonebook()
        self.load_contacts_from_file()

        self.root = root
        self.root.title("Phonebook App")
        self.root.geometry("600x500")  # Increased screen size

        self.style = ttk.Style()
        self.style.configure("TButton", font=("Helvetica", 12))
        self.style.configure("TLabel", font=("Helvetica", 12))
        self.style.configure("TEntry", font=("Helvetica", 12))
        self.style.configure("TFrame", background="#f5f5f5")

        self.main_frame = ttk.Frame(root, padding="10 10 10 10", style="TFrame")
        self.main_frame.pack(fill="both", expand=True)

        self.search_label = ttk.Label(self.main_frame, text="Search:", style="TLabel")
        self.search_label.grid(row=0, column=0, sticky="W", padx=5)
        self.search_entry = ttk.Entry(self.main_frame)
        self.search_entry.grid(row=0, column=1, sticky="EW", padx=5)

        self.search_button = ttk.Button(self.main_frame, text="Search", command=self.search_contacts, style="TButton")
        self.search_button.grid(row=0, column=2, padx=5)

        self.name_label = ttk.Label(self.main_frame, text="Name:", style="TLabel")
        self.name_label.grid(row=1, column=0, sticky="W", padx=5)
        self.name_entry = ttk.Entry(self.main_frame)
        self.name_entry.grid(row=1, column=1, columnspan=2, sticky="EW", padx=5)

        self.phone_label = ttk.Label(self.main_frame, text="Phone:", style="TLabel")
        self.phone_label.grid(row=2, column=0, sticky="W", padx=5)
        self.phone_entry = ttk.Entry(self.main_frame)
        self.phone_entry.grid(row=2, column=1, columnspan=2, sticky="EW", padx=5)

        self.add_button = ttk.Button(self.main_frame, text="Add Contact", command=self.add_contact, style="TButton")
        self.add_button.grid(row=3, column=0, columnspan=3, pady=10)

        self.contacts_frame = ttk.Frame(self.main_frame)
        self.contacts_frame.grid(row=4, column=0, columnspan=3, sticky="nsew")

        self.scrollbar = Scrollbar(self.contacts_frame)
        self.scrollbar.pack(side="right", fill="y")

        self.contacts_listbox = tk.Listbox(self.contacts_frame, yscrollcommand=self.scrollbar.set, font=("Helvetica", 12))
        self.contacts_listbox.pack(fill="both", expand=True)
        self.scrollbar.config(command=self.contacts_listbox.yview)

        self.contacts_listbox.bind('<<ListboxSelect>>', self.show_contact_options)

        self.update_contacts_list()

        self.root.grid_rowconfigure(4, weight=1)
        self.root.grid_columnconfigure(1, weight=1)

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def add_contact(self):
        name = self.name_entry.get()
        phone = self.phone_entry.get()
        if name and phone:
            self.phonebook.add_contact(name, phone)
            messagebox.showinfo("Success", "Contact added successfully!")
            self.name_entry.delete(0, tk.END)
            self.phone_entry.delete(0, tk.END)
            self.update_contacts_list()
        else:
            messagebox.showwarning("Error", "Please enter both name and phone number.")

    def search_contacts(self):
        search_query = self.search_entry.get()
        self.contacts_listbox.delete(0, tk.END)
        contacts = self.phonebook.find_contact(search_query)
        for contact in contacts:
            self.contacts_listbox.insert(tk.END, contact)

    def update_contacts_list(self):
        self.contacts_listbox.delete(0, tk.END)
        contacts = self.phonebook.get_contacts()
        for contact in contacts:
            self.contacts_listbox.insert(tk.END, f"{contact['name']}: {contact['phone']}")

    def show_contact_options(self, event):
        if not self.contacts_listbox.curselection():
            return
        selected_contact = self.contacts_listbox.get(self.contacts_listbox.curselection())
        name, phone = selected_contact.split(': ')
        top = tk.Toplevel(self.root)
        top.title("Edit or Delete Contact")

        ttk.Label(top, text="Name:", style="TLabel").grid(row=0, column=0, sticky="W", padx=5, pady=5)
        name_entry = ttk.Entry(top, font=("Helvetica", 12))
        name_entry.grid(row=0, column=1, padx=5, pady=5)
        name_entry.insert(0, name)
        name_entry.config(state='readonly')

        ttk.Label(top, text="Phone:", style="TLabel").grid(row=1, column=0, sticky="W", padx=5, pady=5)
        phone_entry = ttk.Entry(top, font=("Helvetica", 12))
        phone_entry.grid(row=1, column=1, padx=5, pady=5)
        phone_entry.insert(0, phone)

        def save_changes():
            new_phone = phone_entry.get()
            if new_phone:
                self.phonebook.edit_contact(name, new_phone)
                messagebox.showinfo("Success", "Contact updated successfully!")
                top.destroy()
                self.update_contacts_list()
            else:
                messagebox.showwarning("Error", "Please enter a phone number.")

        def delete_contact():
            self.phonebook.delete_contact(name)
            messagebox.showinfo("Success", "Contact deleted successfully!")
            top.destroy()
            self.update_contacts_list()

        ttk.Button(top, text="Save", command=save_changes, style="TButton").grid(row=2, column=0, pady=10, padx=5)
        ttk.Button(top, text="Delete", command=delete_contact, style="TButton").grid(row=2, column=1, pady=10, padx=5)

        top.geometry("300x150")

    def save_contacts_to_file(self):
        contacts = self.phonebook.get_contacts()
        with open("contacts.json", "w") as f:
            json.dump(contacts, f)

    def load_contacts_from_file(self):
        try:
            with open("contacts.json", "r") as f:
                contacts = json.load(f)
                self.phonebook.load_contacts(contacts)
        except FileNotFoundError:
            pass  # No contacts.json file found, just continue

    def on_closing(self):
        self.save_contacts_to_file()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = PhonebookApp(root)
    root.mainloop()
